In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import random
import os
import cv2
import numpy as np


from scipy.spatial import distance
from insightface.app import FaceAnalysis

In [ ]:
!pip install onnxruntime

In [ ]:
!pip install insightface

# تولید فایل txt

In [ ]:
lfw = '/content/drive/MyDrive/images'
calfw = '/content/drive/MyDrive/calfw/aligned images'
cplfw = '/content/drive/MyDrive/cplfw/aligned images'

file_name = ['lfw.txt', 'calfw.txt', 'cplfw.txt']

dataset_list = [lfw, calfw, cplfw]

for i in range(len(dataset_list)):

  dataset = os.listdir(dataset_list[i])

  if dataset_list[i] == lfw or dataset_list[i] == calfw:

    for j in range(2000):

      rand_number = random.randint(1, 100)
      if rand_number < 50:

        rand_name = random.choice(dataset)
        count = rand_name.split('_')[-1].split('.')[0]
        count = count[2:4]

        if int(count[0]) > 0 :

          img1 = random.randint(1, 9)
          img2 = random.randint(1, 9)

          with open(f'/content/drive/MyDrive/{file_name[i]}', 'a') as txtfile:

            frand_name = rand_name.split('_')[0]
            lrand_name = rand_name.split('_')[1]

            name1 = frand_name + '_' + lrand_name + '_' + '000' + str(img1)  + '.jpg'
            name2 = frand_name + '_' + lrand_name + '_' + '000' + str(img2)  + '.jpg'

            txtfile.write(name1 + '\t' + name2 +'\t' + 'True' + '\n')


        else:
          if int(count[1]) > 1:
            img1 = random.randint(1, int(count[1]))
            img2 = random.randint(1, int(count[1]))

            with open(f'/content/drive/MyDrive/{file_name[i]}', 'a') as txtfile:
              frand_name = rand_name.split('_')[0]
              lrand_name = rand_name.split('_')[1]

              name1 = frand_name + '_' + lrand_name + '_' + '000' + str(img1) + '.jpg'
              name2 = frand_name + '_' + lrand_name + '_' + '000' + str(img2) + '.jpg'

              txtfile.write(name1 +'\t' + name2 +'\t' + 'True' + '\n')

          else:
            with open(f'/content/drive/MyDrive/{file_name[i]}', 'a') as txtfile:

              frand_name = rand_name.split('_')[0]
              lrand_name = rand_name.split('_')[1]

              name = frand_name + '_' + lrand_name  + '_' + '0001' + '.jpg'
              txtfile.write(name +'\t' + name +'\t' + 'True' + '\n')

      else:
        img1 = random.choice(dataset)
        img2 = random.choice(dataset)
        name1 = img1
        name2 = img2
        img1 = img1.split('_')[:2]
        img2 = img2.split('_')[:2]

        with open(f'/content/drive/MyDrive/{file_name[i]}', 'a') as txtfile:
          if img1[0] == img2[0] and img1[1] == img2[1]:
            txtfile.write(name1 + '\t' + name2 + '\t' + 'True' + '\n')
          else:
            txtfile.write(name1 + '\t' + name2 + '\t' + 'Flase' + '\n')
  else:

    for k in range(2000):
      rand_number = random.randint(1,100)
      if rand_number < 50:
        rand_name = random.choice(dataset)
        count = rand_name.split('_')[-1].split('.')[0]

        frand_name = rand_name.split('_')[0]
        lrand_name = rand_name.split('_')[1]

        if int(count) > 1:
          img1 = random.randint(1, int(count))
          img2 = random.randint(1, int(count))

          with open(f'/content/drive/MyDrive/{file_name[i]}', 'a') as txtfile:

            name1 = frand_name + '_' + lrand_name + '_' + str(img1)  + '.jpg'
            name2 = frand_name + '_' + lrand_name + '_' + str(img2)  + '.jpg'
            txtfile.write(name1 + '\t' + name2 + '\t' + 'True' + '\n')

        else:
          with open(f'/content/drive/MyDrive/{file_name[i]}', 'a') as txtfile:
            txtfile.write(frand_name + '_' + lrand_name + '_' + '1' + '.jpg' +
                          '\t' + frand_name + '_' + lrand_name + '_' + '1' + '.jpg' + '\t' + 'True' + '\n')

      else:
        img1 = random.choice(dataset)
        img2 = random.choice(dataset)
        name1 = img1
        name2 = img2
        img1 = img1.split('_')[:2]
        img2 = img2.split('_')[:2]

        with open(f'/content/drive/MyDrive/{file_name[i]}', 'a') as txtfile:
          if img1[0] == img2[0] and img1[1] == img2[1]:
            txtfile.write(name1 + '\t' + name2 + '\t' + 'True' + '\n')
          else:
            txtfile.write(name1 + '\t' + name2 + '\t' + 'Flase' + '\n')

# Face recognition

In [ ]:
app = FaceAnalysis(providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
app.prepare(ctx_id=0, det_size=(640, 640))
print("Model loaded")



lfw = '/content/drive/MyDrive/lfw.txt'

lfw_path = '/content/drive/MyDrive/images/'


accuracy = 0
threshold = 20
FMR = 0
FNMR = 0


dataset_path = [lfw]
path = [lfw_path]

for index, data_path in enumerate(dataset_path):
  with open(data_path, 'r') as myfile:
      for line in myfile:
          columns = line.split()

          image1 = cv2.imread(path[index] + columns[0])
          image2 = cv2.imread(path[index] + columns[1])

          if image1 is not None and image2 is not None:

            faces1 = app.get(image1)
            faces2 = app.get(image2)

            for face1 in faces1:
              for face2 in faces2:
                dist = distance.euclidean(face1['embedding'], face2['embedding'])
                if dist < threshold:
                    print("True" ,'\t', columns[2])
                    if columns[2] == "True":
                      accuracy += 1
                    else:
                      FNMR += 1

                else :
                  print("False", '\t', columns[2])
                  if columns[2] == "False":
                    accuracy += 1
                  else:
                    FMR += 1


acc = (accuracy / 2000) * 100
print("Accuracy : ", acc )

false_match_rate = (FMR / 2000) * 100
flase_not_match_rate = (FNMR / 2000) * 100

print("FMR : ", false_match_rate)
print("FNMR : ", flase_not_match_rate)

